In [108]:
import deepxde as dde
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [109]:
#Paths
sensor_folder = "../data/processed/set3/sensordata"
wear_csv = "../data/processed/set3/merged.csv"

#Load csv
wear_df = pd.read_csv(wear_csv)


In [110]:
#Initialize X and Y
X_all, Y_all = [], []

current_set = None
#Active time since start of current set
cumulative_time = 0.0  

for _, row in wear_df.iterrows():
    #Check which set
    set_number = row["set"]
    if current_set != set_number:
        #New set, so reset cumulative time
        current_set = set_number
        cumulative_time = 0.0
        print(f"Starting new set: {set_number}")
        
    #Find sensor file corresponds to wear level row
    sensor_file = os.path.join(sensor_folder, os.path.basename(row["sensor_window_path"]))
    if not os.path.exists(sensor_file):
        continue

    #Load sensor file
    data = np.load(sensor_file)
    
    #Get sensor data
    sensors = np.stack([data[k] for k in ["accel", "acoustic", "force_x", "force_y", "force_z"]], axis=1)

    #Each file corresponds to sensor data measured between two pictures taken, and for that picture we have one wear level value
    #So, we average these all these sensor values to one mean value that corresponds one wear level (wanted output)
    features = sensors.mean(axis=0)

    #Get duration in one sensor file corresponding to 1 output
    timestamps = pd.to_datetime(data["timestamp"])
    duration = (timestamps[-1] - timestamps[0]).total_seconds()

    #Keep track of cumulative time tool is used
    t_value = cumulative_time
    cumulative_time += duration 


    #Add to X and Y
    X_all.append(np.concatenate([[t_value], features]))
    Y_all.append([row["wear"]]) 

X_all = np.array(X_all)
Y_all = np.array(Y_all)

print("X_all shape:", X_all.shape)
print("Y_all shape:", Y_all.shape)





Starting new set: 3
X_all shape: (104, 6)
Y_all shape: (104, 1)


In [111]:
#Normalization

scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_scaled = scaler_X.fit_transform(X_all)
Y_scaled = scaler_Y.fit_transform(Y_all)

print("X mean:", X_scaled.mean(axis=0))
print("X std:", X_scaled.std(axis=0))
print("Y mean:", Y_scaled.mean(), "Y std:", Y_scaled.std())


X mean: [-2.41900517e-15  5.01094892e-15  1.07819736e-15  7.99307202e-17
  7.28583860e-17  3.56552394e-15]
X std: [1. 1. 1. 1. 1. 1.]
Y mean: -4.2700885562506023e-17 Y std: 0.9999999999999999


In [112]:
print(type(X_scaled[0][0]))

<class 'numpy.float64'>


In [113]:

#DeepXDE needs float32, not float64
X_train = X_scaled.astype(np.float32)
Y_train = Y_scaled.astype(np.float32)

num_sensors = X_train.shape[1]

#Feed-forward network, only 3 hidden layers, should try more (5-10)
net = dde.nn.FNN([num_sensors, 64, 64, 64, 1], "tanh", "Glorot uniform")

#Now using same set for traing and test, should change
data = dde.data.DataSet(X_train, Y_train, X_train, Y_train)

model = dde.Model(data, net)

model.compile("adam", lr=1e-3, metrics=["l2 relative error"])
losshistory, train_state = model.train(epochs=5000, batch_size=32)

Y_pred = model.predict(X_all)
print("Predictions shape:", Y_pred.shape)



Compiling model...
'compile' took 0.003527 s

Training model...

Step      Train loss    Test loss     Test metric   
0         [1.47e+00]    [1.47e+00]    [1.21e+00]    
1000      [1.58e-03]    [1.58e-03]    [3.98e-02]    
2000      [7.07e-04]    [7.07e-04]    [2.66e-02]    
3000      [4.29e-04]    [4.29e-04]    [2.07e-02]    
4000      [1.63e-03]    [1.63e-03]    [4.03e-02]    
5000      [1.44e-04]    [1.44e-04]    [1.20e-02]    

Best model at step 5000:
  train loss: 1.44e-04
  test loss: 1.44e-04
  test metric: [1.20e-02]

'train' took 44.079573 s

Predictions shape: (104, 1)
